In [39]:

import olorenchemengine as oce
from tqdm import tqdm
from tdc.benchmark_group import admet_group
import numpy as np
import pandas as pd



In [40]:
models = {
    'BBB_TDC': '{"BC_class_name": "BaseBoosting", "args": [[{"BC_class_name": "RandomForestModel", "args": [{"BC_class_name": "DescriptastorusDescriptor", "args": ["rdkit2dnormalized"], "kwargs": {}}], "kwargs": {"n_estimators": 1000, "max_features": "log2", "max_depth": null, "criterion": "entropy", "class_weight": null, "bootstrap": true}}, {"BC_class_name": "RandomForestModel", "args": [{"BC_class_name": "DescriptastorusDescriptor", "args": ["morgan3counts"], "kwargs": {}}], "kwargs": {"n_estimators": 1000, "max_features": "log2", "max_depth": null, "criterion": "entropy", "class_weight": null, "bootstrap": true}}, {"BC_class_name": "RandomForestModel", "args": [{"BC_class_name": "OlorenCheckpoint", "args": ["default"], "kwargs": {"num_tasks": 2048}}], "kwargs": {"n_estimators": 1000, "max_features": "log2", "max_depth": null, "criterion": "entropy", "class_weight": null, "bootstrap": true}}]], "kwargs": {"n": 1, "log": true}}',
    'BBB_1040':"{'BC_class_name': 'BaseBoosting', 'args': [[{'BC_class_name': 'RandomForestModel', 'args': [{'BC_class_name': 'DescriptastorusDescriptor', 'args': ['rdkit2dnormalized'], 'kwargs': {'log': True, 'scale': None}}], 'kwargs': {'n_estimators': 1000, 'max_features': 'log2', 'max_depth': None, 'criterion': 'entropy', 'class_weight': None, 'bootstrap': True}}, {'BC_class_name': 'RandomForestModel', 'args': [{'BC_class_name': 'DescriptastorusDescriptor', 'args': ['morgan3counts'], 'kwargs': {'log': True, 'scale': None}}], 'kwargs': {'n_estimators': 1000, 'max_features': 'log2', 'max_depth': None, 'criterion': 'entropy', 'class_weight': None, 'bootstrap': True}}, {'BC_class_name': 'RandomForestModel', 'args': [{'BC_class_name': 'OlorenCheckpoint', 'args': ['default'], 'kwargs': {'num_tasks': 2048, 'log': True}}], 'kwargs': {'n_estimators': 1000, 'max_features': 'log2', 'max_depth': None, 'criterion': 'entropy', 'class_weight': None, 'bootstrap': True}}]], 'kwargs': {'n': 1, 'log': True, 'oof': False, 'nfolds': 5}}",
    'BBB_455': "{'BC_class_name': 'RandomForestModel', 'args': [{'BC_class_name': 'DescriptastorusDescriptor', 'args': ['rdkit2dnormalized'], 'kwargs': {'log': True, 'scale': None}}], 'kwargs': {'n_estimators': 2000, 'max_features': 'log2', 'max_depth': None, 'criterion': 'entropy', 'class_weight': None, 'bootstrap': True}}",
    'BBB_1209':"{'BC_class_name': 'ChemPropModel', 'args': [], 'kwargs': {'epochs': 100, 'dropout_rate': 0.0, 'batch_size': 50, 'lr': 0.001, 'hidden_size': 300, 'depth': 3}}"
}

In [41]:
model_dict = {
    'bbb_base': models["BBB_TDC"],
    'bbb_1040': models["BBB_1040"],
    'bbb_455' : models["BBB_455"],
    'bbb_1209': models["BBB_1209"],
}

In [42]:
def model_from_dict(s):
    s = s.replace("'", "\"").replace("True", "true").replace("False", "false").replace("None", "null")
    model = oce.create_BC(s)
    return model

In [43]:
#EDIT THIS CELL
TASK_NAME = 'bbb_martins'

In [46]:

results_dict = {}
for MODEL in model_dict.keys():
    print(f"__________________________")
    group = admet_group(path = 'data/')
    predictions_list = []
    for seed in tqdm([1, 2, 3, 4, 5]):
        benchmark = group.get(TASK_NAME) 
        predictions = {}
        name = benchmark['name']
        train, test = benchmark['train_val'], benchmark['test']
        model = model_from_dict(model_dict[MODEL])
        model.fit(train['Drug'], train['Y'])
        y_pred_test = model.predict(test['Drug'])
        oce.save(model, MODEL)
        predictions[name] = y_pred_test
        predictions_list.append(predictions)
    results = group.evaluate_many(predictions_list)
    results_dict[MODEL] = results[TASK_NAME]
print(results_dict)

Found local copy...


__________________________


100%|██████████| 5/5 [00:22<00:00,  4.51s/it]
Found local copy...


__________________________


100%|██████████| 5/5 [00:22<00:00,  4.54s/it]
Found local copy...


__________________________


100%|██████████| 5/5 [00:10<00:00,  2.15s/it]
Found local copy...


__________________________


1624it [00:04, 368.53it/s]0<?, ?it/s]
406it [00:01, 379.62it/s]
100%|██████████| 9/9 [00:00<00:00, 101.50it/s]
1624it [00:04, 367.22it/s]1<05:25, 81.34s/it]
406it [00:01, 397.55it/s]
100%|██████████| 9/9 [00:00<00:00, 106.27it/s]
1624it [00:04, 371.62it/s]1<04:01, 80.63s/it]
406it [00:01, 395.17it/s]
100%|██████████| 9/9 [00:00<00:00, 109.17it/s]
1624it [00:04, 369.17it/s]0<02:39, 79.84s/it]
406it [00:01, 396.49it/s]
100%|██████████| 9/9 [00:00<00:00, 109.64it/s]
1624it [00:04, 364.94it/s]9<01:19, 79.46s/it]
406it [00:01, 391.97it/s]
100%|██████████| 5/5 [06:39<00:00, 79.98s/it]

{'bbb_base': [0.922, 0.001], 'bbb_1040': [0.922, 0.001], 'bbb_455': [0.911, 0.0], 'bbb_1209': [0.87, 0.003]}


Baseline Benchmark Results
-------------
|BBB Baseline Model|Error|
|-|-|
|bbb_base |[0.922, 0.001]|
|bbb_1040 |[0.922, 0.001]|
|bbb_455  |[0.911, 0.000]|
|bbb_1209 |[0.870, 0.003]|


